In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np

In [8]:
train_df = pd.read_csv('train.csv')
valid_df = pd.read_csv('valid.csv')
test_df = pd.read_csv('test.csv')

In [9]:
train_df.head(5)

,Brightness,Contrast,Mean,Variance,Skewness,Kurtosis,Entropy,Energy,Absolute Moment k=1,Absolute Moment k=2,...,Information Measure I,Information Measure II,Maximal Correlation Coefficient,Short-run Emphasis,Long-run Emphasis,Gray-level Nonuniformity,Difference of Entropy,Second Largest Eigenvalue,Label,Image
0,146.146736,81.409801,146.146736,6627.555637,-0.186254,-1.798750,6.974621,0.011098,146.146736,117.858576,...,1.0,0.0,0.010016,1.0,0.0,0.000006,4.469887,0.015243,Hibiscus,IMG_20241029_131633_jpg.rf.a3f2d7fa633ae0536e8...
1,147.130032,67.900449,147.130032,4610.471012,-0.399389,-1.412432,7.179110,0.010070,147.130032,117.151824,...,1.0,0.0,0.014951,1.0,0.0,0.000007,4.732864,0.020229,Hibiscus,IMG_20241029_131640_jpg.rf.30a12f9b36a51bb91c4...
2,148.789408,73.502056,148.789408,5402.552291,-0.262159,-1.708594,6.882111,0.011922,148.789408,114.736928,...,1.0,0.0,0.012129,1.0,0.0,0.000007,3.966379,0.018935,Hibiscus,IMG_20241029_131646_jpg.rf.c743c72ae1c3b403d16...
3,131.630976,76.952406,131.630976,5921.672749,-0.136606,-1.796559,6.932319,0.010645,131.630976,116.349472,...,1.0,0.0,0.008145,1.0,0.0,0.000005,4.478143,0.012706,Hibiscus,IMG_20241029_131651_jpg.rf.203f579e95e3f696a72...
4,150.084368,60.223102,150.084368,3626.821986,-0.411602,-1.391630,6.861676,0.013588,150.084368,116.505328,...,1.0,0.0,0.017838,1.0,0.0,0.000009,4.438945,0.019402,Hibiscus,IMG_20241029_131711_jpg.rf.c3aee7f34bec54a8830...


In [11]:
X_train, y_train = train_df.drop(columns=['Image', 'Label']), train_df['Label']
X_valid, y_valid = valid_df.drop(columns=['Image', 'Label']), valid_df['Label']
X_test, y_test = test_df.drop(columns=['Image', 'Label']), test_df['Label']

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [13]:
results = {}

Given a model it will evaluate with the help of the earlier dataset

In [14]:
def evaluate_model(name, model, X_train, y_train, X_valid, y_valid, X_test, y_test):
    print(f"\nTraining {name} model...")
    model.fit(X_train, y_train)

    y_valid_pred = model.predict(X_valid)
    print(f"\nValidation Performance of {name}:")
    print(confusion_matrix(y_valid, y_valid_pred))
    print(classification_report(y_valid, y_valid_pred))

    y_test_pred = model.predict(X_test)
    print(f"\nTest Performance of {name}:")
    print(confusion_matrix(y_test, y_test_pred))
    print(classification_report(y_test, y_test_pred))

### 1. Support Vector Machine

In [15]:
svm_model = SVC(kernel='linear', C=1)
evaluate_model("SVM", svm_model, X_train, y_train, X_valid, y_valid, X_test, y_test)


Training SVM model...

Validation Performance of SVM:
[[ 1  0  0  0  0  0  0  0  0  1]
 [ 0  5  0  0  0  1  0  1  2  0]
 [ 0  0  4  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  1  0  1]
 [ 0  0  0  0  3  0  0  0  1  0]
 [ 0  0  0  0  1  1  0  1  0  0]
 [ 1  1  0  0  1  0  3  1  0  1]
 [ 0  0  0  0  0  0  0  3  0  0]
 [ 1  0  0  0  0  1  0  0 59  0]
 [ 0  0  3  0  0  4  0  0  0  0]]
              precision    recall  f1-score   support

      Ajwain       0.33      0.50      0.40         2
      Almond       0.83      0.56      0.67         9
      Ashoka       0.57      0.80      0.67         5
   DrumStick       0.00      0.00      0.00         2
    Fittonia       0.60      0.75      0.67         4
    Hibiscus       0.14      0.33      0.20         3
   JackFruit       1.00      0.38      0.55         8
       Mango       0.43      1.00      0.60         3
        Neem       0.94      0.97      0.95        61
     Parijat       0.00      0.00      0.00         7

    accuracy      

e:\!VNIT College\A. Pattern Recognition\Leaf\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\!VNIT College\A. Pattern Recognition\Leaf\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\!VNIT College\A. Pattern Recognition\Leaf\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

### 2. Parzen Window (Linear Classifier)

In [16]:
from sklearn.naive_bayes import GaussianNB
parzen_model = GaussianNB()
evaluate_model("Parzen Windows (Gaussian Naive Bayes)", parzen_model, X_train, y_train, X_valid, y_valid, X_test, y_test)



Training Parzen Windows (Gaussian Naive Bayes) model...

Validation Performance of Parzen Windows (Gaussian Naive Bayes):
[[ 1  0  0  0  0  1  0  0  0  0]
 [ 2  0  0  0  2  0  1  3  1  0]
 [ 0  0  3  0  1  1  0  0  0  0]
 [ 0  1  0  0  1  0  0  0  0  0]
 [ 0  1  0  0  3  0  0  0  0  0]
 [ 0  0  0  0  1  2  0  0  0  0]
 [ 3  0  0  0  2  0  3  0  0  0]
 [ 0  0  0  0  0  0  1  2  0  0]
 [ 1  0  2  5  1  3  3 33 13  0]
 [ 2  0  2  0  0  2  0  1  0  0]]
              precision    recall  f1-score   support

      Ajwain       0.11      0.50      0.18         2
      Almond       0.00      0.00      0.00         9
      Ashoka       0.43      0.60      0.50         5
   DrumStick       0.00      0.00      0.00         2
    Fittonia       0.27      0.75      0.40         4
    Hibiscus       0.22      0.67      0.33         3
   JackFruit       0.38      0.38      0.38         8
       Mango       0.05      0.67      0.10         3
        Neem       0.93      0.21      0.35        61
     

e:\!VNIT College\A. Pattern Recognition\Leaf\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\!VNIT College\A. Pattern Recognition\Leaf\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\!VNIT College\A. Pattern Recognition\Leaf\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

### 3. k-Nearest Neighbor

In [20]:
knn_model = KNeighborsClassifier(n_neighbors=8)
evaluate_model("KNN", knn_model, X_train, y_train, X_valid, y_valid, X_test, y_test)


Training KNN model...

Validation Performance of KNN:
[[ 1  0  0  0  1  0  0  0  0  0]
 [ 1  1  0  0  0  0  0  0  7  0]
 [ 1  0  2  0  0  0  0  1  1  0]
 [ 0  1  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  4  0  0  0  0  0]
 [ 0  1  0  0  0  2  0  0  0  0]
 [ 2  1  1  0  1  0  2  1  0  0]
 [ 0  0  1  0  0  0  0  2  0  0]
 [ 1  0  0  0  0  0  0  1 59  0]
 [ 1  0  3  0  0  2  0  0  0  1]]
              precision    recall  f1-score   support

      Ajwain       0.14      0.50      0.22         2
      Almond       0.25      0.11      0.15         9
      Ashoka       0.29      0.40      0.33         5
   DrumStick       0.00      0.00      0.00         2
    Fittonia       0.67      1.00      0.80         4
    Hibiscus       0.50      0.67      0.57         3
   JackFruit       1.00      0.25      0.40         8
       Mango       0.40      0.67      0.50         3
        Neem       0.87      0.97      0.91        61
     Parijat       1.00      0.14      0.25         7

    accuracy      

e:\!VNIT College\A. Pattern Recognition\Leaf\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\!VNIT College\A. Pattern Recognition\Leaf\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\!VNIT College\A. Pattern Recognition\Leaf\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

### 4. Random Forest 

In [19]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
evaluate_model("Random Forest", rf_model, X_train, y_train, X_valid, y_valid, X_test, y_test)


Training Random Forest model...

Validation Performance of Random Forest:
[[ 0  0  0  0  1  0  0  0  0  1]
 [ 1  3  0  0  0  0  0  0  5  0]
 [ 0  0  4  0  0  0  0  1  0  0]
 [ 0  0  0  1  0  0  0  1  0  0]
 [ 0  0  0  0  4  0  0  0  0  0]
 [ 0  0  0  0  1  2  0  0  0  0]
 [ 1  2  0  0  2  0  3  0  0  0]
 [ 0  0  0  0  0  1  0  2  0  0]
 [ 0  0  0  0  0  0  0  1 60  0]
 [ 0  0  2  0  0  1  0  0  0  4]]
              precision    recall  f1-score   support

      Ajwain       0.00      0.00      0.00         2
      Almond       0.60      0.33      0.43         9
      Ashoka       0.67      0.80      0.73         5
   DrumStick       1.00      0.50      0.67         2
    Fittonia       0.50      1.00      0.67         4
    Hibiscus       0.50      0.67      0.57         3
   JackFruit       1.00      0.38      0.55         8
       Mango       0.40      0.67      0.50         3
        Neem       0.92      0.98      0.95        61
     Parijat       0.80      0.57      0.67         7

e:\!VNIT College\A. Pattern Recognition\Leaf\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\!VNIT College\A. Pattern Recognition\Leaf\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\!VNIT College\A. Pattern Recognition\Leaf\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr